# XY-FZ35: electronic load in custom build housing

## Overview

The **XY-FZ35** is a small and very inexpensive electronic load with a maximum **load power of 35 W** and **load current of 5 A**. Unlike considerably more expensive devices with a wide range of functions, the XY-FZ35 offers "Constant Current (CC)" as its only function. Therefore, it is suitable, for example, to investigate both the performance and the response of various protection circuits of wall mounted power supply outputs, mobile power banks or conditioning batteries.

![Front view of XY-FZ35 build in custom housing](images/XY-FZ35_electronic_load_front.jpeg)  
Front view of XY-FZ35 build in custom housing

The XY-FZ35 is not a ready-to-use device, but is intended for snap-in mounting for installation in a front panel or enclosed housing. My assembly looks like this:

![Inside top view of XY-FZ35 build in custom housing](images/XY-FZ35_electronic_load_inside_view_w_description.png)  
Inside top view of XY-FZ35 build in custom housing

## Technical data

- Snap-in panel device
- Input voltage: 5.0 - 30.0 V DC (reverse polarity protected)
- Load voltage: 1.5 - 25.0 V DC (reverse polarity protected)
- Load current: 0.00 - 5.00 A with 0.01 A resolution
- Load power: 35 W at maximum
- Current regulation: ±(1% + 3 digits)
- Voltage regulation: ±(0.5% + 1 digit)
- Over voltage protection (OVP): default 25.2 V (adjustable)
- Over current protection (OCP): default 5.1O A (adjustable)
- Over power protection (OPP): default 35.5 W (adjustable)
- Over temperature protection (OTP): ~ 80 °C (fixed)

<span style="color:green">**Note:**</span> The display flashes and shows the error code e.g. `OVP` or `OCP`.

- Low voltage protection (LVP): default 1.5 V (adjustable)

<span style="color:green">**Note:**</span> Important for battery discharge tests: Setting the LVP value can prevent battery from deep discharge.


- Operating Temperature: -40 to ~85 °C

The fan starts automatically when the load power is greater than 10 W or the temperature is greater than 40 °C.

<span style="color:green">**Note:**</span> My FZ35 starts the fan when the load current is greater than or equal to 1.5 A - regardless of the load power or the current temperature. Maybe it is a firmware bug in my version of the FZ35.

## Literature:

- [Data sheet XY-FZ35](https://m.media-amazon.com/images/I/B1rtWZuqjcS.pdf)
- [XY-FZ35 - Inexpensive Electronic Load](https://community.element14.com/challenges-projects/project14/test-instrumentation/b/blog/posts/xy-fz35---inexpensive-electronic-load)
- [makerspacelt / fz35-cli](https://github.com/makerspacelt/fz35-cli)
- [XY-FZ25/35 Communication Description](https://github.com/ah01/fz35/blob/master/communication.md)
- [XY-FZ25 & XY-FZ35 Electronic Load Control Program](https://github.com/yellobyte/ElectronicLoad-Control-XY-FZ35)
- [FZ35 Adjustable Electronic Load Questions](https://www.eevblog.com/forum/testgear/fz35-adjustabe-electronic-load-questions/)

# Serial interface for controlling the XY-FZ35

This HTML style override aligns tables to the left side of all subsequent Markdown cells:

In [9]:
%%html
<style>
  table {margin-left: 0 !important;}
</style>

Following description was inspired by and adapted from [https://github.com/ah01/fz35/blob/master/communication.md](https://github.com/ah01/fz35/blob/master/communication.md).

## Electric characteristics

- TTL level communication (5 V level, there is a XL1509-5.0 buck voltage regulator on board)

<span style="color:red">**Warnings:**</span>
- TX and RX pins are connected directly to MCU pins without any protection.
- There is **NO** galvanic isolation between communication interface, power supply or load input.

For easier use, I added a serial to USB converter with **CP2102** chipset.

## Serial connection parameters

| Serial parameter | Setting  |
|------------------|----------|
| Baud Rate        | 9600 bps |
| Data bits        | 8        |
| Stop bits        | 1        |
| Parity           | None     |
| Flow control     | None     |

## Protocol

- serial master-slave communication
- commands are to be sent **without** any line ending (like `CR`, `LF` or both)
- replies ending with `CRLF`

### Commands

Short overview:

| Command     | Reply      | Note                              |
|-------------|------------|-----------------------------------|
| `start`     | S/F        | Start periodic measurement upload |
| `stop`      | S/F        | Stop upload                       |
| `on`        | S/F        | Turn on load function             |
| `off`       | S/F        | Turn off load function            |
| `x.xxA`     | S/F        | Set load current                  |
| `LVP:xx.x`  | S/F        | Set low voltage protection        |
| `OVP:xx.x`  | S/F        | Set over voltage protection       |
| `OCP:x.xx`  | S/F        | Set over current protection       |
| `OPP:xx.xx` | S/F        | Set over power Protection         |
| `OAH:x.xxx` | S/F        | Set maximum capacity              |
| `OHP:xx:xx` | S/F        | Set maximum discharge time        |
| `read`      | parameters | Read product parameter settings   |

Description:
- **LVP:** (Low Voltage Protection) If the voltage drops below a set value then the load will turn itself off. This is important for discharge tests on batteries in order to protect the battery from deep discharge.
- **OAH**: (Maximum Discharge Capacity) When the load is turned on it calculates the accumulated discharge capacity (in Ah) and turns itself off when a set value has been reached. This feature too is for protecting the battery when doing discharge tests.
- **OHP**: (Maximum Discharge Time) When the discharge time reaches a set period of time than the load will turn itself off. Important for discharge tests.
- **OVP**: (Over Voltage Protection) If the voltage is greater then a set value the load will turn itself off.
- **OCP**: (Over Current Protection) If the current is greater then a set value the load will turn itself off.
- **OPP**: (Over Power Protection) If the power it absorbs gets greater then a set value then the load will turn itself off.

<span style="color:green">**Note:**</span> Some alarms (**OPP**, **OAH**, **OHP**) can't be cleared via serial communication. In those cases the On/Off Button on the device itself must be pressed to end the alarm and get the device operational again. Message Boxes will tell you if that's the case.

### S/F Replies (success/fail)

Most commands have a reply just `success` or `fail`.

Fail usualy means wrong format or a value out of range. Especially the format is tricky - you need to send exact same decimal digits as required (including leading and ending zeros) and also **no line ending**!

Examples: `OPP:05.00` will work, but `OPP:5` or `OPP:5.0` or `OPP:05.00<CR><LF>` will not.

<span style="color:green">**Note:**</span> My FZ35 returns in success case `sucess` (sic). But [ah01](https://github.com/ah01/fz35/blob/master/communication.md) stated that there are some implementations of communication library that respond `success` with correct spelling. So maybe there are some more FW versions out there.

### Parameters reply

For `read` command the device will reply with current setting in following format:

```
OVP:xx.x, OCP:x.xx, OPP:xx.xx, LVP:xx.x,OAH:x.xxx,OHP:xx:xx<CR><LF>
```

<span style="color:green">**Note:**</span> Spaces are correct.

### Measurement upload

After `start` command the device will start sending current measurement every 1 second with following format:

```
xx.xxV,x.xA,x.xxxAh,xx:xx<CR><LF>
```

# Test program to retrieve measurements

## Load globally used libraries and set global variables

In [1]:
# import classes from external python files
import importlib.util

import serial
import time
import pandas as pd
import datetime

In [8]:
ENABLED_CSV_LOGGING = True

SERIAL_PORT = "/dev/ttyUSB0"

# after writing commands a pause is needed
CMD_WRITE_PAUSE = 0.2

## Helper functions

This is a helper function to add new rows (a list) to measurement data frame:

In [3]:
def dataframe_add_row(df=None, row=[]):
    if (df is None):
        return
    
    # add a row
    df.loc[-1] = row
    
    # shift the index
    df.index = df.index + 1
    
    # reset the index of dataframe and avoid the old index being added as a column
    df.reset_index(drop=True, inplace=True)

With following function a time string in the format 'hh:mm' is converted to float:

In [4]:
def convert_time_str2float(str_time='0:00'):
    # split time string by ':'
    hours, minutes = str_time.split(':')

    # convert hours and minutes to seconds (and hours by dividing with 3600)
    f_time_hours = datetime.timedelta(hours=int(hours), minutes=int(minutes)).total_seconds() / 3600

    # round to 3 decimal places
    f_time_hours = float("{:.3f}".format(f_time_hours))
    
    return f_time_hours

## Configure TSV logging

With the help of the external class `Log2CSV` the measurement data are stored in a TSV file (i.e. a tab-delimited CSV file).

To protect the SD card from excessive write accesses, the data is first buffered in memory by the `Log2CSV` class and written to the TSV file at specified time intervals.

In [5]:
if ENABLED_CSV_LOGGING:
    # class Log2CSV has to imported via importlib due to different path of notebook and class file
    spec = importlib.util.spec_from_file_location("Log2CSV", "./Log2CSV_class.py")
    log2csv_class = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(log2csv_class)
    
    # every 60 seconds the internal data frame buffer should be written to the CSV file
    CSV_LOGGING_INTERVALL = 60

## Control the XY-FZ35

Initialize the serial connection to XY-FZ35:

In [6]:
try:
    serial = serial.Serial(port=SERIAL_PORT, baudrate=9600, timeout=2)

except serial.SerialException:
    print("Could not open serial port '{0}'!".format(SERIAL_PORT))
    raise

Set the load current:

In [17]:
f_load_current = 0.8

str_current = '{:.2f}'.format(f_load_current) + 'A'
serial.write(str_current.encode('utf-8'))
time.sleep(CMD_WRITE_PAUSE)

# read setting success
print(serial.read_all())

b'sucess\r\n'


Start the electronic load to discharge the power source:

In [18]:
serial.write('on'.encode('utf-8'))
time.sleep(CMD_WRITE_PAUSE)

# read setting success
print(serial.read_all())

b'sucess\r\n'


## Main function

Following function is the main function to retrieve the measurements from XY-FZ35.

In [20]:
# Set Protection
OVP = 'OVP:25.2'        # over voltage
OCP = 'OCP:5.00'        # over current
OPP = 'OPP:35.10'       # over power
LVP = 'LVP:01.5'        # low voltage

LOOP_PAUSE = 0.95

list_csv_header = ['Measuring Time [h]',
                   'Discharge Runtime [h]',
                   'Voltage [V]',
                   'Current [A]',
                   'Capacity [Ah]']

if ENABLED_CSV_LOGGING:
    str_csv_file = time.strftime('./data_files/' + '%Y-%m-%d_%H_%M') + '_XY-FZ35_auto_measurement.tsv'
    
    # create csv logging object
    csvLogger = log2csv_class.Log2CSV(str_csv_file, CSV_LOGGING_INTERVALL, list_csv_header)

# stop periodic measurement upload
serial.write('stop'.encode('utf-8'))
time.sleep(CMD_WRITE_PAUSE)

serial.write(LVP.encode('utf-8'))
time.sleep(CMD_WRITE_PAUSE)
serial.write(OVP.encode('utf-8'))
time.sleep(CMD_WRITE_PAUSE)
serial.write(OCP.encode('utf-8'))
time.sleep(CMD_WRITE_PAUSE)
serial.write(OPP.encode('utf-8'))
time.sleep(CMD_WRITE_PAUSE)

#print(serial.read_all())      # read setting success
serial.flushInput()
time.sleep(CMD_WRITE_PAUSE)

# read parameters
serial.write('read'.encode('utf-8'))
time.sleep(CMD_WRITE_PAUSE)
print(serial.read_all().decode('utf-8'))
time.sleep(CMD_WRITE_PAUSE)

# start periodic measurement upload
serial.write('start'.encode('utf-8'))
time.sleep(CMD_WRITE_PAUSE)
#print(serial.read_all())
serial.flushInput()
time.sleep(CMD_WRITE_PAUSE)

# get starting time
time_start_sec = float("{:.2f}".format(time.time()))

while True:
    try:
        # get time relative to starting time and round to 3 decimals
        timestamp_hours = float("{:.3f}".format( (time.time() - time_start_sec) / 3600) )

        values_row = [timestamp_hours]

        raw_data = serial.read_all()

        # decode it - make it a string
        str_data = raw_data.decode('utf-8')

        # check if string is complete
        while not ('\r' in str_data and '\n' in str_data):
            #print('string is NOT ok ... read again and join')

            # if not: read again and concatenate
            raw_data_residual = serial.read_all()

            str_data = str_data + raw_data_residual.decode('utf-8')
            time.sleep(CMD_WRITE_PAUSE)

        # strip newlines from string
        str_data = str_data.strip()

        # split string to list
        list_values = str_data.split(',')

        for idx, val in enumerate(list_values):
            if idx == 0 or idx == 1:
                # cut units 'V' or 'A'
                list_values[idx] = float(list_values[idx][:-1])
            elif idx == 2:
                # cut unit 'Ah'
                list_values[idx] = float(list_values[idx][:-2])
            elif idx == 3:
                # convert time from hh:mm to float
                list_values[idx] = convert_time_str2float(list_values[idx])

            values_row.append(list_values[idx])

            # re-order the list to make 'Discharge Runtime' the 2. element
            list_neworder = [1, 3, 4, 5, 2]
            values_row_sorted = [x for i, x in sorted(zip(list_neworder, values_row))]
        
        if ENABLED_CSV_LOGGING:
            # log row to csv file
            csvLogger.log_data(values_row_sorted)

        print('Measuring Time: {} h, Discharge Runtime: {} h, Voltage: {} V, Current: {} A, Capacity: {} Ah'
              .format(values_row_sorted[0],
                      values_row_sorted[1],
                      values_row_sorted[2],
                      values_row_sorted[3],
                      values_row_sorted[4]))

        time.sleep(LOOP_PAUSE)
        
    except:
        print("Keyboard Interrupt ^C detected.")
        print("Bye.")
        
        # stop periodic measurement upload
        serial.write('stop'.encode('utf-8'))
        time.sleep(CMD_WRITE_PAUSE)
        serial.flushInput()

        break

OVP:25.2, OCP:5.00, OPP:35.10, LVP:01.5,OAH:0.000,OHP:00:00

Measuring Time: 0.0 h, Discharge Runtime: 0.117 h, Voltage: 5.0 V, Current: 0.8 A, Capacity: 0.103 Ah
Measuring Time: 0.001 h, Discharge Runtime: 0.117 h, Voltage: 4.99 V, Current: 0.8 A, Capacity: 0.103 Ah
Measuring Time: 0.001 h, Discharge Runtime: 0.117 h, Voltage: 4.99 V, Current: 0.8 A, Capacity: 0.104 Ah
Measuring Time: 0.001 h, Discharge Runtime: 0.117 h, Voltage: 4.98 V, Current: 0.8 A, Capacity: 0.104 Ah
Measuring Time: 0.001 h, Discharge Runtime: 0.117 h, Voltage: 4.99 V, Current: 0.8 A, Capacity: 0.104 Ah
Measuring Time: 0.002 h, Discharge Runtime: 0.117 h, Voltage: 4.99 V, Current: 0.8 A, Capacity: 0.104 Ah
Measuring Time: 0.002 h, Discharge Runtime: 0.117 h, Voltage: 4.99 V, Current: 0.8 A, Capacity: 0.105 Ah
Measuring Time: 0.002 h, Discharge Runtime: 0.117 h, Voltage: 4.99 V, Current: 0.8 A, Capacity: 0.105 Ah
Measuring Time: 0.003 h, Discharge Runtime: 0.117 h, Voltage: 4.99 V, Current: 0.8 A, Capacity: 0.105 

Stop the electronic load to stop discharging:

In [21]:
serial.write('off'.encode('utf-8'))
time.sleep(CMD_WRITE_PAUSE)

# read setting success
print(serial.read_all())

b'sucess\r\n'
